# Procesamiento y clustering de Vecindarios en Toronto

## Instalación e importación de liberías necesarias

In [1]:
!pip install geopy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


## Importación y preparación de datos

Cada frame de datos,
-   Datos de los vecindarios extistentes data_cleaned_toronto.csv
-   Datos geoespaciales de los vecindarios existentes Geoespatial_Coordinate.csv

fue cargado al notebook y se le hizo un merge para poder tener toda la información en un solo dataFrame

In [3]:
data_geo = pd.read_csv("./Geospatial_Coordinates.csv") # Cargamos datos geoespaciales
data_desc = pd.read_csv("./data_cleaned_toronto.csv") # cargamos datos, CP
data_geo = data_geo.rename(columns={"Postal Code": 'Postalcode'}) #Homologamos la columna para hacer el merge
data_geo.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
data_desc.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern
1,M1B,Scarborough,Rouge
2,M1C,Scarborough,Rouge Hill
3,M1C,Scarborough,Port Union
4,M1C,Scarborough,Highland Creek


In [4]:
data_geo.dtypes

Postalcode     object
Latitude      float64
Longitude     float64
dtype: object

In [5]:
data_desc.dtypes

Postalcode      object
Borough         object
Neighborhood    object
dtype: object

Definimos total_data como la unión de toda la información que tenemos para cada vecindario

In [72]:
total_data = data_desc.merge(data_geo, how='left', on='Postalcode')

In [8]:
total_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
1,M1B,Scarborough,Rouge,43.806686,-79.194353
2,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
3,M1C,Scarborough,Port Union,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497


Filtramos la data para solo obtener los vecindarios que contengan en el campo Borough la palabra "Toronto"

In [73]:
filter_data = total_data[total_data['Borough'].str.contains(r' Toronto')]
print(filter_data['Borough'].unique())
filter_data.head()

['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


,Postalcode,Borough,Neighborhood,Latitude,Longitude
72,M4K,East Toronto,The Danfort West,43.679557,-79.352188
73,M4K,East Toronto,Riverdale,43.679557,-79.352188
74,M4L,East Toronto,India Bazaar,43.668999,-79.315572
75,M4L,East Toronto,The Beaches West,43.668999,-79.315572
76,M4M,East Toronto,Studio District,43.659526,-79.340923


Imprimimos los vecindarios que se encuentran en los municipios filtrados

In [10]:
print(filter_data['Neighborhood'].sort_values(ascending=True))

117                                         Harbord
81                                  Summerhill East
75                                 The Beaches West
152                              The Junction South
100                                        Adelaide
125                                  Barthurst Quay
97                                      Berczy Park
142                                        Brockton
121                                        CN Tower
90                                      Cabbagetown
98                               Central Bay Street
119                                       Chinatown
136                                        Christie
91                             Church and Wellesley
107                                  Commerce Court
80                                       Davisville
78                                 Davisville North
87                                        Deer Park
106                                 Design Exchange
138         

Realizamos una descripción de los vecindarios encontrados para el filtro de selección

In [11]:
filter_data.describe()

,Latitude,Longitude
count,69.000000,69.000000
mean,43.659985,-79.395788
std,0.021660,0.032405
min,43.628947,-79.484450
25%,43.647927,-79.405678
50%,43.654260,-79.394420
75%,43.669542,-79.381576
max,43.728020,-79.315572


Visualizamos los vecindarios encontrados con la palabra "Toronto"

In [12]:
filter_data['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(filter_data['Borough'].unique()),
        filter_data.shape[0])
    )

The dataframe has 4 boroughs and 69 neighborhoods.


Obtenemos las coordenadas de la dirección principal donde aparecerá nuestro mapa

In [14]:
address = "Toronto, Ontario, Canadá"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.6534817, -79.3839347.


Generamos el mapa con los puntos de los vecindarios generados

In [15]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(filter_data['Latitude'], filter_data['Longitude'], filter_data['Borough'], filter_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Clustering de Vecindarios Toronto

Ahora, para poder hacer el clustering de vecindarios, utilizaremos Foursquare API para extraer información acerca de los lugares que podemos encontrar en cada vecindario

In [16]:
#Generamos nuestras credenciales
headers = {"Accept":"application/json",
          "Authorization":"Your API KEY here"}

In [17]:
# función para extraer la categoria del sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
def getNearbyVenues(names, latitudes, longitudes, headers, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search"
        search_query=''
        params = {
        "ll": str(lat)+","+str(lng),
        "radius": 500,
        "limit": 50,
        }
            
        # solicitud GET
        response = requests.request("GET", url, params=params, headers=headers)
        results = response.json()
        results = results['results']

        # regresa solo información relevante de cada sitio cercano
        for venue in results:
                try:
                    venues_list.append([name.strip(), lat,lng, 
                                        venue['name'], 
                                        venue['geocodes']['main']['latitude'],
                                        venue['geocodes']['main']['longitude'],
                                        venue['categories']['name']])
                except Exception as e:
                    venues_list.append([name.strip(), lat,lng, 
                                        venue['name'], 
                                        venue['geocodes']['main']['latitude'],
                                        venue['geocodes']['main']['longitude'],
                                        'Not Especified'])
    print(venues_list)
    #nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Obtengamos los 50 sitios de cada vecindario en un radio de 500 mts

In [25]:
# Corremos la funcion para cada vecindario en la tabla
toronto_venues = getNearbyVenues(names=filter_data['Neighborhood'],
                                   latitudes=filter_data['Latitude'],
                                   longitudes=filter_data['Longitude'],headers= headers)

The Danfort West
Riverdale
India Bazaar
 The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
 Summerhill East
Moore Park
Rathnelly
Summerhill West
South Hill
Forest Hill
Deer Park
Rosedale
St. James Town
Cabbagetown
Church and Wellesley
Regent Park
Harbourfront
Garden District
Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond
Adelaide
King
Harbourfront East
Union Station
Toronto Islands
Toronto Dominion Centre
Design Exchange
Commerce Court
Victoria Hotel
The Annex
North Midtown
Yorkville
University of Toronto
 Harbord
Kensington Market
Chinatown
Grange Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Barthurst Quay
South Niagara
Island airport
Stn A PO Boxes25 The EsplanadeEnclave of M5E
First Canadian Place
Underground city
Christie
Dufferin
Dovercourt Village
Trinity
Little Portugal
Parkdale Village
Brockton
Exhibition Place
 The Junction South
High Park
Roncesvalles
Parkdale
Swansea
Runnymede
[['The Danfort West',

Echemos un vistazo a la data extraída

In [26]:
print(toronto_venues.shape)
toronto_venues.tail()

(3105, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3100,Runnymede,43.651571,-79.48445,Pizza Pizza,43.649172,-79.484363,Pizzeria
3101,Runnymede,43.651571,-79.48445,Dr. Wat Lisa,43.649181,-79.483477,Dentist
3102,Runnymede,43.651571,-79.48445,Bloor West Smiles,43.649478,-79.484787,Dentist
3103,Runnymede,43.651571,-79.48445,Dark Horse,43.649430,-79.483143,American Restaurant
3104,Runnymede,43.651571,-79.48445,Pizzaiolo,43.649174,-79.483569,Pizzeria


Eliminamos los registros de lugares encontrados que no contienen una categoria

In [43]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Not Especified']
print(toronto_venues.shape)

(2953, 7)


Veamos el numero de lugares encontrados para cada vecindario

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,50,50,50,50,50,50
Barthurst Quay,6,6,6,6,6,6
Berczy Park,49,49,49,49,49,49
Brockton,48,48,48,48,48,48
CN Tower,6,6,6,6,6,6
Cabbagetown,46,46,46,46,46,46
Central Bay Street,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Christie,45,45,45,45,45,45


¿Cuantas categorias unicas de lugares encontramos?

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 326 uniques categories.


Visualizamos los tipos de lugares encontrados

In [46]:
toronto_venues['Venue Category'].unique()

array(['Elementary School', 'Pizzeria', 'Arts and Entertainment',
       'Ice Cream Parlor', 'Bubble Tea Shop', 'Burger Joint',
       'Greek Restaurant', 'Bakery', 'Convenience Store',
       'Physical Therapy Clinic', 'Party Supply Store',
       'Health Food Store', 'Bar', 'Cheese Store', 'Grocery Store',
       'Park', 'Toy Store', 'Alternative Medicine Clinic', 'Gift Store',
       'Organic Grocery', 'Clothing Store', 'Juice Bar', 'Bookstore',
       'Mobile Phone Store', 'Vegan and Vegetarian Restaurant',
       'Restaurant', "Children's Clothing Store",
       'Fruit and Vegetable Store', 'Sushi Restaurant', 'Thai Restaurant',
       'Persian Restaurant', 'Church', 'Farmers Market', 'Breakfast Spot',
       'Fast Food Restaurant', 'Business and Professional Services',
       'Electronics Store', 'Automotive Repair Shop', 'Police Station',
       'Computers and Electronics Retail',
       'Heating, Ventilating and Air Conditioning Contractor',
       'Architecture Firm', 'Pet Sup

### Analizar cada vecindario

In [47]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.tail()

,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,Alternative Medicine Clinic,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Art School,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Audiovisual Service,Auditorium,Automotive Repair Shop,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bike Trail,Bistro,Board Store,Bookstore,Boutique,Breakfast Spot,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Café,Camera Store,Campground,Candy Store,Capitol Building,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Caterer,Cemetery,Charity,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chiropractor,Chocolate Store,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College and University,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Book Store,Community Center,Community and Government,Computer Repair Service,Computers and Electronics Retail,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Costume Store,Creperie,Cuban Restaurant,Culinary School,Dance Studio,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Doors and Windows Contractor,Drugstore,Dry Cleaner,Education,Electrician,Electronics Store,Elementary School,Embassy or Consulate,Engineer,Entertainment Agency,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Falafel Restaurant,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food Court,Food Truck,Food and Beverage Retail,Food and Beverage Service,Fountain,Framing Store,French Restaurant,Fried Chicken Joint,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gastropub,General Contractor,Gift Store,Golf Course,Gourmet Store,Government Building,Government Department,Greek Restaurant,Grocery Store,Hair Salon,Halal Restaurant,Harbor or Marina,Hardware Store,Health Food Store,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",High School,Hiking Trail,Historic and Protected Site,Hobby Store,Home Appliance Store,Home Health Care Service,Home Improvement Service,Home Inspection,Hospital,Hot Dog Joint,Hotel Bar,Hotpot Restaurant,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Language School,Latin American Restaurant,Law Office,Lebanese Restaurant,Legal Service,Library,Lighthouse,Liquor Store,Loans Agency,Locksmith,Lodging,Lounge,Luggage Store,Mac and Cheese Joint,Machine Shop,Management Consultant,Martial Arts Dojo,Massage Clinic,Media Agency,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mental Health Clinic,Mental Health Service,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Mobile Phone Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Motorcycle Repair Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Night Club,Non-Profit Organization,Noodle Restaurant,Office Building,Office Supply Store,Ophthalmologist,Optometrist,Organic Grocery,Organization,Orthopedic Surgeon,Other Great Outdoors,Packaging Supply Store,Painter,Park,Party Supply Store,Performing Arts Venue,Persian Restaurant,Pest Control Service,Pet Service,Pet Supplies Store,Photographer,Physical Therapy Clinic

In [48]:
# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,Alternative Medicine Clinic,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Art School,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Audiovisual Service,Auditorium,Automotive Repair Shop,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bike Trail,Bistro,Board Store,Bookstore,Boutique,Breakfast Spot,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Café,Camera Store,Campground,Candy Store,Capitol Building,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Caterer,Cemetery,Charity,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chiropractor,Chocolate Store,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College and University,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Book Store,Community Center,Community and Government,Computer Repair Service,Computers and Electronics Retail,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Costume Store,Creperie,Cuban Restaurant,Culinary School,Dance Studio,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Doors and Windows Contractor,Drugstore,Dry Cleaner,Education,Electrician,Electronics Store,Elementary School,Embassy or Consulate,Engineer,Entertainment Agency,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Falafel Restaurant,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food Court,Food Truck,Food and Beverage Retail,Food and Beverage Service,Fountain,Framing Store,French Restaurant,Fried Chicken Joint,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gastropub,General Contractor,Gift Store,Golf Course,Gourmet Store,Government Building,Government Department,Greek Restaurant,Grocery Store,Hair Salon,Halal Restaurant,Harbor or Marina,Hardware Store,Health Food Store,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",High School,Hiking Trail,Historic and Protected Site,Hobby Store,Home Appliance Store,Home Health Care Service,Home Improvement Service,Home Inspection,Hospital,Hot Dog Joint,Hotel Bar,Hotpot Restaurant,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Language School,Latin American Restaurant,Law Office,Lebanese Restaurant,Legal Service,Library,Lighthouse,Liquor Store,Loans Agency,Locksmith,Lodging,Lounge,Luggage Store,Mac and Cheese Joint,Machine Shop,Management Consultant,Martial Arts Dojo,Massage Clinic,Media Agency,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mental Health Clinic,Mental Health Service,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Mobile Phone Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Motorcycle Repair Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Night Club,Non-Profit Organization,Noodle Restaurant,Office Building,Office Supply Store,Ophthalmologist,Optometrist,Organic Grocery,Organization,Orthopedic Surgeon,Other Great Outdoors,Packaging Supply Store,Painter,Park,Party Supply Store,Performing Arts Venue,Persian Restaurant,Pest Control Service,Pet Service,Pet Supplies Store,Photographer,Physical T

In [49]:
toronto_onehot.shape

(2953, 327)

#### Agrupemos las filas por barrios tomando la média de la frecuancia de la ocurrencia de cada categoría

In [50]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,Alternative Medicine Clinic,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Art School,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Audiovisual Service,Auditorium,Automotive Repair Shop,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bike Trail,Bistro,Board Store,Bookstore,Boutique,Breakfast Spot,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Café,Camera Store,Campground,Candy Store,Capitol Building,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Caterer,Cemetery,Charity,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chiropractor,Chocolate Store,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College and University,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Book Store,Community Center,Community and Government,Computer Repair Service,Computers and Electronics Retail,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Costume Store,Creperie,Cuban Restaurant,Culinary School,Dance Studio,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Doors and Windows Contractor,Drugstore,Dry Cleaner,Education,Electrician,Electronics Store,Elementary School,Embassy or Consulate,Engineer,Entertainment Agency,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Falafel Restaurant,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food Court,Food Truck,Food and Beverage Retail,Food and Beverage Service,Fountain,Framing Store,French Restaurant,Fried Chicken Joint,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gastropub,General Contractor,Gift Store,Golf Course,Gourmet Store,Government Building,Government Department,Greek Restaurant,Grocery Store,Hair Salon,Halal Restaurant,Harbor or Marina,Hardware Store,Health Food Store,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",High School,Hiking Trail,Historic and Protected Site,Hobby Store,Home Appliance Store,Home Health Care Service,Home Improvement Service,Home Inspection,Hospital,Hot Dog Joint,Hotel Bar,Hotpot Restaurant,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Language School,Latin American Restaurant,Law Office,Lebanese Restaurant,Legal Service,Library,Lighthouse,Liquor Store,Loans Agency,Locksmith,Lodging,Lounge,Luggage Store,Mac and Cheese Joint,Machine Shop,Management Consultant,Martial Arts Dojo,Massage Clinic,Media Agency,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mental Health Clinic,Mental Health Service,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Mobile Phone Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Motorcycle Repair Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Night Club,Non-Profit Organization,Noodle Restaurant,Office Building,Office Supply Store,Ophthalmologist,Optometrist,Organic Grocery,Organization,Orthopedic Surgeon,Other Great Outdoors,Packaging Supply Store,Painter,Park,Party Supply Store,Performing Arts Venue,Persian Restaurant,Pest Control Service,Pet Service,Pet Supplies Store,Photographer,Physical T

Confirmemos el nuevo tamaño

In [51]:
toronto_grouped.shape

(68, 327)

### Veamos cada vecindario con su top5 de lugares

In [52]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                venue  freq
0          Steakhouse  0.06
1  Italian Restaurant  0.04
2                Deli  0.04
3    Salad Restaurant  0.04
4                Café  0.04


----Barthurst Quay----
                         venue  freq
0               Car Dealership  0.17
1     Home Improvement Service  0.17
2        Sports and Recreation  0.17
3  Historic and Protected Site  0.17
4             Harbor or Marina  0.17


----Berczy Park----
                    venue  freq
0                     Pub  0.04
1              Steakhouse  0.04
2  Arts and Entertainment  0.04
3         Ophthalmologist  0.04
4       Convenience Store  0.04


----Brockton----
                                venue  freq
0  Business and Professional Services  0.08
1                     Office Building  0.06
2                             Dentist  0.06
3           Telecommunication Service  0.06
4                        Media Agency  0.04


----CN Tower----
                         venue  freq
0              

### Pongamos esa información en el dataframe

Primero escribamos una función para ordenar los sitios en orden descendente.

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada barrio.

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Italian Restaurant,Deli,Salad Restaurant,Café,"Cafe, Coffee, and Tea House",Sushi Restaurant,Music Venue,Monument,Thai Restaurant
1,Barthurst Quay,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue
2,Berczy Park,Pub,Steakhouse,Arts and Entertainment,Ophthalmologist,Convenience Store,Miscellaneous Store,Park,"Cafe, Coffee, and Tea House",French Restaurant,Street Food Gathering
3,Brockton,Business and Professional Services,Office Building,Dentist,Telecommunication Service,Media Agency,Drugstore,Coffee Shop,Technology Business,Grocery Store,Medical Center
4,CN Tower,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue


In [55]:
print(neighborhoods_venues_sorted)

                                    Neighborhood   
0                                       Adelaide  \
1                                 Barthurst Quay   
2                                    Berczy Park   
3                                       Brockton   
4                                       CN Tower   
5                                    Cabbagetown   
6                             Central Bay Street   
7                                      Chinatown   
8                                       Christie   
9                           Church and Wellesley   
10                                Commerce Court   
11                                    Davisville   
12                              Davisville North   
13                                     Deer Park   
14                               Design Exchange   
15                            Dovercourt Village   
16                                      Dufferin   
17                              Exhibition Place   
18          

In [56]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',axis= 1)
toronto_grouped_clustering.head()

,ATM,Accounting and Bookkeeping Service,Advertising Agency,Afghan Restaurant,African Restaurant,Alternative Medicine Clinic,American Restaurant,Antique Store,Architecture Firm,Art Gallery,Art School,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Audiovisual Service,Auditorium,Automotive Repair Shop,Automotive Service,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball,Bathroom Contractor,Beer Bar,Beer Store,Bicycle Store,Bike Trail,Bistro,Board Store,Bookstore,Boutique,Breakfast Spot,Bridal Store,Bubble Tea Shop,Buffet,Building and Land Surveyor,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Café,Camera Store,Campground,Candy Store,Capitol Building,Car Dealership,Car Wash and Detail,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Caterer,Cemetery,Charity,Cheese Store,Children's Clothing Store,Chinese Restaurant,Chiropractor,Chocolate Store,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College and University,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Book Store,Community Center,Community and Government,Computer Repair Service,Computers and Electronics Retail,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Costume Store,Creperie,Cuban Restaurant,Culinary School,Dance Studio,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Dim Sum Restaurant,Diner,Dining and Drinking,Discount Store,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Doors and Windows Contractor,Drugstore,Dry Cleaner,Education,Electrician,Electronics Store,Elementary School,Embassy or Consulate,Engineer,Entertainment Agency,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Falafel Restaurant,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Film Studio,Financial Planner,Financial Service,Fish Market,Fish and Chips Shop,Flower Store,Food Court,Food Truck,Food and Beverage Retail,Food and Beverage Service,Fountain,Framing Store,French Restaurant,Fried Chicken Joint,Fruit and Vegetable Store,Furniture and Home Store,Garden,Garden Center,Gastropub,General Contractor,Gift Store,Golf Course,Gourmet Store,Government Building,Government Department,Greek Restaurant,Grocery Store,Hair Salon,Halal Restaurant,Harbor or Marina,Hardware Store,Health Food Store,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",High School,Hiking Trail,Historic and Protected Site,Hobby Store,Home Appliance Store,Home Health Care Service,Home Improvement Service,Home Inspection,Hospital,Hot Dog Joint,Hotel Bar,Hotpot Restaurant,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Interior Designer,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Landmarks and Outdoors,Landscaper and Gardener,Language School,Latin American Restaurant,Law Office,Lebanese Restaurant,Legal Service,Library,Lighthouse,Liquor Store,Loans Agency,Locksmith,Lodging,Lounge,Luggage Store,Mac and Cheese Joint,Machine Shop,Management Consultant,Martial Arts Dojo,Massage Clinic,Media Agency,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mental Health Clinic,Mental Health Service,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Mobile Phone Store,Molecular Gastronomy Restaurant,Monument,Motorcycle Dealership,Motorcycle Repair Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Night Club,Non-Profit Organization,Noodle Restaurant,Office Building,Office Supply Store,Ophthalmologist,Optometrist,Organic Grocery,Organization,Orthopedic Surgeon,Other Great Outdoors,Packaging Supply Store,Painter,Park,Party Supply Store,Performing Arts Venue,Persian Restaurant,Pest Control Service,Pet Service,Pet Supplies Store,Photographer,Physical Therapy Clinic

In [57]:
toronto_grouped_clustering.dtypes

ATM                                                       float64
Accounting and Bookkeeping Service                        float64
Advertising Agency                                        float64
Afghan Restaurant                                         float64
African Restaurant                                        float64
Alternative Medicine Clinic                               float64
American Restaurant                                       float64
Antique Store                                             float64
Architecture Firm                                         float64
Art Gallery                                               float64
Art School                                                float64
Arts and Crafts Store                                     float64
Arts and Entertainment                                    float64
Asian Restaurant                                          float64
Audiovisual Service                                       float64
Auditorium

## Vecindarios Agrupados

Ejecutemos *k*-means para agrupar los barrios en 5 agrupaciones.

In [58]:
# establecer el número de agrupaciones
kclusters = 5

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_

c:\Users\MarleneElizabethLope\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([2, 0, 2, 1, 0, 2, 3, 3, 1, 2, 3, 2, 1, 4, 3, 1, 1, 1, 3, 4, 2, 3,
       2, 1, 2, 0, 1, 1, 0, 3, 2, 0, 1, 2, 1, 1, 1, 1, 1, 0, 4, 1, 2, 2,
       1, 1, 4, 2, 4, 0, 2, 2, 2, 1, 4, 4, 1, 1, 2, 1, 3, 2, 2, 3, 2, 2,
       3, 1])

Generemos un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.

In [59]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

toronto_merged = filter_data

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(6) # revisar las ultimas columnas

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,M4K,East Toronto,The Danfort West,43.679557,-79.352188,2.0,Greek Restaurant,Ice Cream Parlor,Fruit and Vegetable Store,Thai Restaurant,Bubble Tea Shop,Burger Joint,Sushi Restaurant,Bakery,Gift Store,Bookstore
73,M4K,East Toronto,Riverdale,43.679557,-79.352188,2.0,Greek Restaurant,Ice Cream Parlor,Fruit and Vegetable Store,Thai Restaurant,Bubble Tea Shop,Burger Joint,Sushi Restaurant,Bakery,Gift Store,Bookstore
74,M4L,East Toronto,India Bazaar,43.668999,-79.315572,1.0,Pizzeria,Park,Convenience Store,Café,Architecture Firm,Business and Professional Services,Pet Supplies Store,Grocery Store,"Heating, Ventilating and Air Conditioning Cont...",Healthcare Clinic
75,M4L,East Toronto,The Beaches West,43.668999,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,M4M,East Toronto,Studio District,43.659526,-79.340923,2.0,Drugstore,Automotive Repair Shop,Bar,Bakery,Clothing Store,Retail,Painter,Fast Food Restaurant,Tattoo Parlor,Sushi Restaurant
77,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1.0,Business and Strategy Consulting Office,Architecture Firm,Publisher,Construction,Tennis Club,Management Consultant,Legal Service,Bakery,Organization,Healthcare Clinic


Finalmente visualicemos las agrupaciones resultantes

In [60]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array =['blue','green','red','yellow','black']

# añadir marcadores al mapa
markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    try:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=colors_array[int(cluster)],
            fill=True,
            fill_color=colors_array[int(cluster)],
            fill_opacity=0.7).add_to(map_clusters)
    except Exception as e:
        print(e)
       
map_clusters

cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer


### Mostramos la info de los vecindarios agrupados en cada cluster

### Para el Cluster 0

In [63]:
cluster0 = toronto_merged[toronto_merged['Cluster Labels'] == 0]
cluster0.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,M5V,Downtown Toronto,CN Tower,43.628947,-79.39442,0.0,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue
122,M5V,Downtown Toronto,King and Spadina,43.628947,-79.39442,0.0,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue
123,M5V,Downtown Toronto,Railway Lands,43.628947,-79.39442,0.0,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue
124,M5V,Downtown Toronto,Harbourfront West,43.628947,-79.39442,0.0,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue
125,M5V,Downtown Toronto,Barthurst Quay,43.628947,-79.39442,0.0,Car Dealership,Home Improvement Service,Sports and Recreation,Historic and Protected Site,Harbor or Marina,Landmarks and Outdoors,ATM,Motorcycle Repair Shop,New American Restaurant,Music Venue


### Para el Cluster 1

In [64]:
cluster1 = toronto_merged[toronto_merged['Cluster Labels'] == 1]
cluster1.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,M4L,East Toronto,India Bazaar,43.668999,-79.315572,1.0,Pizzeria,Park,Convenience Store,Café,Architecture Firm,Business and Professional Services,Pet Supplies Store,Grocery Store,"Heating, Ventilating and Air Conditioning Cont...",Healthcare Clinic
77,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1.0,Business and Strategy Consulting Office,Architecture Firm,Publisher,Construction,Tennis Club,Management Consultant,Legal Service,Bakery,Organization,Healthcare Clinic
78,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1.0,Pet Service,Business and Professional Services,Community and Government,Public and Social Service,Landscaper and Gardener,Tutoring Service,Restaurant,Painter,Playground,Hair Salon
79,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1.0,Clothing Store,Church,Cosmetics Store,Health Food Store,Children's Clothing Store,Park,Audiovisual Service,Hot Dog Joint,"Cafe, Coffee, and Tea House",Sporting Goods Retail
82,M4T,Central Toronto,Moore Park,43.689574,-79.383160,1.0,Elementary School,Business and Strategy Consulting Office,Financial Planner,Business and Professional Services,Architecture Firm,Church,Tennis Court,Organization,Interior Designer,Translation Service


### Para el Cluster 2

In [67]:
cluster2 = toronto_merged[toronto_merged['Cluster Labels'] == 2]
cluster2.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,M4K,East Toronto,The Danfort West,43.679557,-79.352188,2.0,Greek Restaurant,Ice Cream Parlor,Fruit and Vegetable Store,Thai Restaurant,Bubble Tea Shop,Burger Joint,Sushi Restaurant,Bakery,Gift Store,Bookstore
73,M4K,East Toronto,Riverdale,43.679557,-79.352188,2.0,Greek Restaurant,Ice Cream Parlor,Fruit and Vegetable Store,Thai Restaurant,Bubble Tea Shop,Burger Joint,Sushi Restaurant,Bakery,Gift Store,Bookstore
76,M4M,East Toronto,Studio District,43.659526,-79.340923,2.0,Drugstore,Automotive Repair Shop,Bar,Bakery,Clothing Store,Retail,Painter,Fast Food Restaurant,Tattoo Parlor,Sushi Restaurant
80,M4S,Central Toronto,Davisville,43.704324,-79.388790,2.0,Flower Store,Candy Store,Jewelry Store,Toy Store,Italian Restaurant,Bakery,Optometrist,New American Restaurant,Electronics Store,Locksmith
89,M4X,Downtown Toronto,St. James Town,43.667967,-79.367675,2.0,Pizzeria,Bakery,Coffee Shop,Café,Indian Restaurant,Diner,Restaurant,Asian Restaurant,Night Club,Bar


### Para el Cluster 3

In [66]:
cluster3 = toronto_merged[toronto_merged['Cluster Labels'] == 3]
cluster3.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3.0,"Cafe, Coffee, and Tea House",Coffee Shop,Deli,Pizzeria,Ramen Restaurant,Café,Japanese Restaurant,Asian Restaurant,Italian Restaurant,Fast Food Restaurant
105,M5K,Downtown Toronto,Toronto Dominion Centre,43.647177,-79.381576,3.0,Deli,Restaurant,"Cafe, Coffee, and Tea House",Bakery,Café,Diner,Bar,Retail,Cocktail Bar,Shopping Mall
106,M5K,Downtown Toronto,Design Exchange,43.647177,-79.381576,3.0,Deli,Restaurant,"Cafe, Coffee, and Tea House",Bakery,Café,Diner,Bar,Retail,Cocktail Bar,Shopping Mall
107,M5L,Downtown Toronto,Commerce Court,43.648198,-79.379817,3.0,Restaurant,"Cafe, Coffee, and Tea House",Bakery,Café,Coffee Shop,Deli,Retail,Arts and Entertainment,Bistro,Pub
108,M5L,Downtown Toronto,Victoria Hotel,43.648198,-79.379817,3.0,Restaurant,"Cafe, Coffee, and Tea House",Bakery,Café,Coffee Shop,Deli,Retail,Arts and Entertainment,Bistro,Pub


### Para el Cluster 4

In [65]:
cluster4 = toronto_merged[toronto_merged['Cluster Labels'] == 4]
cluster4.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M4V,Central Toronto,Rathnelly,43.686412,-79.400049,4.0,Dentist,Physician,Sports and Recreation,Healthcare Clinic,Shopping Plaza,Advertising Agency,General Contractor,Non-Profit Organization,Office Building,Real Estate Agency
84,M4V,Central Toronto,Summerhill West,43.686412,-79.400049,4.0,Dentist,Physician,Sports and Recreation,Healthcare Clinic,Shopping Plaza,Advertising Agency,General Contractor,Non-Profit Organization,Office Building,Real Estate Agency
85,M4V,Central Toronto,South Hill,43.686412,-79.400049,4.0,Dentist,Physician,Sports and Recreation,Healthcare Clinic,Shopping Plaza,Advertising Agency,General Contractor,Non-Profit Organization,Office Building,Real Estate Agency
86,M4V,Central Toronto,Forest Hill,43.686412,-79.400049,4.0,Dentist,Physician,Sports and Recreation,Healthcare Clinic,Shopping Plaza,Advertising Agency,General Contractor,Non-Profit Organization,Office Building,Real Estate Agency
87,M4V,Central Toronto,Deer Park,43.686412,-79.400049,4.0,Dentist,Physician,Sports and Recreation,Healthcare Clinic,Shopping Plaza,Advertising Agency,General Contractor,Non-Profit Organization,Office Building,Real Estate Agency


### Finalmente para los vecindarios que no se pudieron agrupar

(Aqui podiamos borrarlos, pero decidí conservarlos para hacer evidente que no pudieron agruparse porque no se extrajo de manera exitosa la información del vecindario desde Foursquare)

In [70]:
nan_venues = toronto_merged[toronto_merged['Cluster Labels'].isnull()]
nan_venues.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,M4L,East Toronto,The Beaches West,43.668999,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,M4T,Central Toronto,Summerhill East,43.689574,-79.383160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,M5S,Downtown Toronto,Harbord,43.662696,-79.400049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,M6P,West Toronto,The Junction South,43.661608,-79.464763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
